In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 139.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.

In [3]:
!pip install rouge_score

In [10]:
from datasets import load_dataset


dataset = load_dataset("Ekinezya2025/ilac-soru-cevap")
dataset=dataset["train"]


In [5]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Read the medical information and provide a clear, simplified, and accurate answer to the question so that a non-medical person can understand it. If safety warnings or important usage instructions are relevant, include them clearly.

### Input:
{}

### Response:
"""

max_seq_length = 512
dtype = "float16"
load_in_4bit = True

torch.cuda.empty_cache()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Ekinezya2025/LlamaModel",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.5: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2025.8.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [10]:
import json, random, csv, os
from statistics import mean
from tqdm.auto import tqdm


def compare_on_dataset_1k(dataset, model, tokenizer, alpaca_prompt):

    results = []
    from rouge_score import rouge_scorer
    from sentence_transformers import util

    scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)

    for i in tqdm(range(min(1000, len(dataset))), desc="Evaluating"):
        question = dataset[i]["question"]
        real_answer = dataset[i]["answer"]

        model_answer = generate_answer(question, model, tokenizer, alpaca_prompt)


        scores = scorer.score(real_answer, model_answer)


        emb = embedder.encode([real_answer, model_answer], convert_to_tensor=True, normalize_embeddings=True)
        semantic_sim = util.cos_sim(emb[0], emb[1]).item()

        results.append({
            "idx": i,
            "question": question,
            "real_answer": real_answer,
            "model_answer": model_answer,
            "rouge1": scores["rouge1"].fmeasure,
            "rouge2": scores["rouge2"].fmeasure,
            "rougeL": scores["rougeL"].fmeasure,
            "semantic_similarity": semantic_sim
        })
    return results


results = compare_on_dataset_1k(dataset, model, tokenizer, alpaca_prompt)


avg_r1 = mean(r["rouge1"] for r in results)
avg_r2 = mean(r["rouge2"] for r in results)
avg_rL = mean(r["rougeL"] for r in results)
avg_sem = mean(r["semantic_similarity"] for r in results)

print(f"\nAverages on {len(results)} examples:")
print(f"  ROUGE-1 F1: {avg_r1:.4f}")
print(f"  ROUGE-2 F1: {avg_r2:.4f}")
print(f"  ROUGE-L F1: {avg_rL:.4f}")
print(f"  Semantic Cosine: {avg_sem:.4f}")



Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]


Averages on 1000 examples:
  ROUGE-1 F1: 0.2621
  ROUGE-2 F1: 0.1476
  ROUGE-L F1: 0.2269
  Semantic Cosine: 0.5288

Kaydedildi: /kaggle/working/eval_200_results.csv
